In [ ]:
import os

os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import cv2
from PIL import Image


In [ ]:
from watch_recognition.predictors import TFLiteDetector
from watch_recognition.predictors import KPPredictor
from watch_recognition.predictors import HandPredictor
from watch_recognition.predictors import RotationPredictor
keypoint_model = '../cloud_train/local-training-output/heatmap-regression_098DB017/models/keypoint/efficientnetb0-unet-96-hands/run_1636479122.611962/'
sm_model_path = "../models/effnet-b3-FPN-(160, 160)-935-weighted-jl/F661D8C2"

hand_predictor = HandPredictor(sm_model_path)
kp_predictor = KPPredictor(keypoint_model)
rp = RotationPredictor("./models/angle-classifier/efficientnetb0-8/run_1635014448.999021/")


In [ ]:

detector = TFLiteDetector("./models/detection/efficientdet_lite0/run_1633100188.371347/model.tflite")

In [ ]:
from tqdm import tqdm
import dataclasses
from watch_recognition.models import points_to_time

file = Path('../IMG_1200_720p.mov')
assert file.exists()
cap = cv2.VideoCapture(str(file))

# Read until video is completed
print((int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
out = cv2.VideoWriter(
    "outpy_13.mp4",
    cv2.VideoWriter_fourcc('A','V','C','1'),
    cap.get(cv2.CAP_PROP_FPS) - 10,
    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))),
)
crop_padding = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))//2
print(crop_padding)
use_angle_model = False
frame_id = 0
with tqdm(total=393) as pbar:
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame[crop_padding:-crop_padding, :]
            new_results = []
            with Image.fromarray(frame) as pil_img:
                try:
                    bboxes = detector.predict(pil_img)
                    for box in bboxes:
                        pred_center, pred_top = kp_predictor.predict_from_image_and_bbox(pil_img, box, rotation_predictor=rp)
                        frame = pred_center.draw_marker(frame, thickness=2)
                        frame = pred_top.draw_marker(frame, thickness=2)
                        minute_and_hour, other = hand_predictor.predict_from_image_and_bbox(pil_img, box, pred_center)
                        if minute_and_hour:
                            pred_minute, pred_hour = minute_and_hour
                            read_hour, read_minute = points_to_time(
                                pred_center, pred_hour.end, pred_minute.end, pred_top
                            )
                            frame = pred_minute.draw(frame, thickness=3)
                            frame = pred_minute.end.draw_marker(frame, thickness=2)
                            frame = pred_hour.draw(frame, thickness=5)
                            frame = pred_hour.end.draw_marker(frame, thickness=2)

                            time = f"{read_hour:.0f}:{read_minute:.0f}"
                            new_results.append(dataclasses.replace(box, name=time))
                        else:
                            new_results.append(dataclasses.replace(box, name="???"))

                        for line in other:
                            frame = line.draw(frame, thickness=1, color=(255, 0, 0))
                except Exception as e:
                    print(e, frame_id)
                    raise e
            for box in new_results:
                frame = box.draw(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            out.write(frame)
        # Break the loop
        else:
            break
        frame_id += 1
        pbar.update(1)
        # if frame_id > 2:
        #     break


cap.release()
out.release()
cv2.destroyAllWindows()
frame_id